In [7]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from shallowmind.src.model import ModelInterface
from shallowmind.src.data import DataInterface
from shallowmind.api.infer import prepare_inference
from shallowmind.src.utils import load_config
import pytorch_lightning as pl
import torch
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, average_precision_score

pl.seed_everything(42)

def config_function(config):
    config.data['train']['data_root'] = config.data['train']['data_root'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
    config.data['val']['data_root'] = config.data['val']['data_root'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
    config.data['test']['data_root'] = config.data['test']['data_root'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')

    config.data['train']['temp_save_folder'] = config.data['train']['temp_save_folder'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
    config.data['val']['temp_save_folder'] = config.data['val']['temp_save_folder'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
    config.data['test']['temp_save_folder'] = config.data['test']['temp_save_folder'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
    return config

def load_model_data(config_path, ckpt_path,confing_function=None):
    data_module, model = prepare_inference(config_path, ckpt_path, confing_function=confing_function)
    data_module.setup()
    test_loader = data_module.test_dataloader()
    return model, test_loader

def model_inference(model, test_loader, device='cuda', metrics=['accuracy', 'recall', 'precision', 'f1', 'roc_auc', 'pr_auc']):
    # inference
    model = model.to(device)
    model = model.eval()
    data_table = test_loader.dataset.data_table
    predictions = []
    labels = []
    for batch_id,d in enumerate(test_loader):
        with torch.no_grad():
            data = {'seq': d[0]['seq'].to(device),}
            label = d[1].to(device)
            out = model.model.forward(data, label) 
            if len(out) == 2:
                loss, pred = out[0], out[1]
            elif len(out) == 3:
                loss, pred, att = out[0], out[1], out[2]
            pred = pred.argmax(dim=1)
            predictions.append(pred.cpu().numpy())
            labels.append(label.cpu().numpy())
    predictions = np.concatenate(predictions)
    labels = np.concatenate(labels)
    res = pd.DataFrame(columns=metrics, index=[0])
    for metric in metrics:
        if metric == 'accuracy':
            res[metric] = accuracy_score(labels, predictions)
        elif metric == 'recall':
            res[metric] = recall_score(labels, predictions)
        elif metric == 'precision':
            res[metric] = precision_score(labels, predictions)
        elif metric == 'f1':
            res[metric] = f1_score(labels, predictions)
        elif metric == 'roc_auc':
            res[metric] = roc_auc_score(labels, predictions)
        elif metric == 'pr_auc':
            res[metric] = average_precision_score(labels, predictions)
    return res


work_dir = '/data/xiaowjia/Friends_fNIRS/work_dir'
save_path = '/data/xiaowjia/Friends_fNIRS/output/results.csv'
if os.path.exists(save_path):
    res_table = pd.read_csv(save_path)
else:
    res_table = pd.DataFrame(columns=['experiment_path', 'dataset_type', 'dataset_label', 'dataset_name', 'base_lr', 'batch_size', 'model_name', 'model_type', 'num_heads', 'depth', 'num_rules']+['accuracy', 'recall', 'precision', 'f1', 'roc_auc', 'pr_auc'])

experiments_paths = [f for f in os.listdir(work_dir)]
for experiment_path in tqdm(experiments_paths, total=len(experiments_paths)):
    try:
        if not os.path.exists(os.path.join(work_dir, experiment_path, 'ckpts','last.ckpt')): 
            print(f'no ckpt: {experiment_path}')
            continue
        if experiment_path in res_table['experiment_path'].values: 
            print(f'skip: {experiment_path}')
            continue
        print(f'processing: {experiment_path}')
        config_poth = [f for f in os.listdir(os.path.join(work_dir, experiment_path)) if f.endswith('.py')][0]
        config_path = os.path.join(work_dir, experiment_path, config_poth)
        ckpt_path = [f for f in os.listdir(os.path.join(work_dir, experiment_path, 'ckpts')) if not f.startswith('last')]
        ckpt_path = sorted(ckpt_path, key=lambda x: float(x.split('=')[-1].split('.')[0]), reverse=True)[0]
        ckpt_path = os.path.join(work_dir, experiment_path, 'ckpts', ckpt_path)
        # read config details

        # load config
        model, test_loader = load_model_data(config_path, ckpt_path, confing_function=config_function)
        config = load_config(config_path)
        res = model_inference(model, test_loader)

        # add para to res
        res['experiment_path'] = experiment_path

        dataset_type = config['dataset_para']['type']
        res['dataset_type'] = dataset_type

        dataset_label = config['dataset_para']['label']
        res['dataset_label'] = dataset_label

        dataset_name = config['dataset_name']
        res['dataset_name'] = dataset_name

        base_lr = config['base_lr']
        res['base_lr'] = base_lr

        batch_size = config['batch_size']
        res['batch_size'] = batch_size

        model_name = config['model']['type']
        res['model_name'] = model_name

        model_type = config['model_para']['encoder_type']
        res['model_type'] = model_type

        num_heads = config['model_para']['num_heads']
        res['num_heads'] = num_heads

        depth = config['model_para']['depth']
        res['depth'] = depth

        try:
            num_rules = config['model_para']['num_rules']
            res['num_rules'] = num_rules
        except:
            res['num_rules'] = 0
        
        res_table = pd.concat([res_table, res], axis=0)
        res_table.to_csv(save_path, index=False)
    except Exception as e:
        print(f'failed: {experiment_path}: {e}')
        continue

Global seed set to 42
  0%|          | 0/3475 [00:00<?, ?it/s]

skip: AFTF-ds=PictureClass_l-d=1-nh=5-r=40-bs=128-ls=0.015-label=hand
skip: FTF-ds=PictureClass_n-d=2-nh=5-r=40-bs=128-ls=0.00015-label=relationship
no ckpt: AFTF-ds=PictureClass_l-d=3-nh=19-r=5-bs=128-ls=5e-05-label=relationship
skip: FTF-ds=PictureRating_d-d=3-nh=8-r=1-bs=256-ls=0.0015-label=hand
skip: FTF-ds=PictureClass_l-d=1-nh=19-r=1-bs=256-ls=0.00015-label=relationship
skip: FTF-ds=PictureRating_l-d=2-nh=11-r=10-bs=256-ls=0.015-label=relationship
skip: FTF-ds=PictureClass_n-d=1-nh=8-r=1-bs=128-ls=0.03-label=relationship
no ckpt: baseline_TFe-ds=PictureClass_t-d=2-nh=5-bs=128-ls=5e-05-label=relationship
processing: baseline_TFe-ds=PictureRating_t-d=1-nh=3-bs=128-ls=0.015-label=hand
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform


  0%|          | 9/3475 [00:12<1:22:17,  1.42s/it]

skip: FTF-ds=PictureRating_d-d=1-nh=5-r=10-bs=256-ls=0.00015-label=emotion
skip: FTF-ds=PictureRating_t-d=1-nh=11-r=1-bs=128-ls=0.003-label=relationship
skip: FTF-ds=PictureClass_t-d=1-nh=5-r=5-bs=128-ls=0.015-label=relationship
no ckpt: AFTF-ds=PictureRating_l-d=1-nh=11-r=1-bs=128-ls=0.015-label=relationship
skip: AFTF-ds=PictureClass_l-d=1-nh=19-r=40-bs=128-ls=5e-05-label=relationship
processing: baseline_TFe-ds=PictureRating_t-d=2-nh=11-bs=128-ls=0.0015-label=relationship
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform


  0%|          | 15/3475 [00:34<2:25:14,  2.52s/it]

no ckpt: AFTF-ds=PictureClass_l-d=1-nh=5-r=1-bs=128-ls=0.0015-label=hand
no ckpt: AFTF-ds=PictureClass_l-d=2-nh=19-r=1-bs=128-ls=5e-05-label=emotion
skip: FTF-ds=PictureClass_t-d=3-nh=5-r=5-bs=128-ls=0.0015-label=relationship
no ckpt: baseline_TFe-ds=PictureRating_t-d=1-nh=3-bs=128-ls=0.003-label=emotion
skip: AFTF-ds=PictureClass_l-d=2-nh=5-r=40-bs=128-ls=0.003-label=emotion
processing: baseline_TFe-ds=PictureClass_t-d=1-nh=19-bs=128-ls=0.015-label=relationship
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform
The pipeline is not setup, will use identity transform


  1%|          | 21/3475 [00:48<2:19:45,  2.43s/it]

no ckpt: AFTF-ds=PictureRating_l-d=1-nh=3-r=5-bs=128-ls=0.015-label=hand
no ckpt: AFTF-ds=PictureClass_l-d=2-nh=19-r=40-bs=128-ls=5e-05-label=relationship
skip: FTF-ds=PictureRating_d-d=2-nh=8-r=10-bs=256-ls=5e-05-label=hand
no ckpt: FTF-ts=0.3-nr=3-d=2-nh=8bs=128-ls=0.00015-label=relationship
skip: FTF-ds=PictureRating_d-d=1-nh=8-r=40-bs=256-ls=0.00015-label=relationship
skip: FTF-ds=PictureRating_l-d=3-nh=11-r=40-bs=256-ls=0.003-label=relationship
skip: FTF-ds=PictureRating_l-d=1-nh=11-r=5-bs=256-ls=0.00015-label=emotion
skip: FTF-ds=PictureRating_l-d=3-nh=11-r=10-bs=256-ls=0.015-label=relationship
skip: FTF-ds=PictureClass_d-d=3-nh=5-r=40-bs=256-ls=0.015-label=relationship
skip: FTF-ds=PictureRating_n-d=3-nh=5-r=1-bs=128-ls=0.0015-label=relationship
no ckpt: AFTF-ds=PictureClass_l-d=1-nh=19-r=1-bs=128-ls=0.03-label=hand
skip: FTF-ds=PictureClass_n-d=1-nh=5-r=40-bs=128-ls=0.03-label=relationship
skip: AFTF-ds=PictureRating_l-d=1-nh=11-r=10-bs=128-ls=5e-05-label=relationship
no ckpt: 

  1%|          | 37/3475 [00:49<1:15:54,  1.32s/it]


KeyboardInterrupt: 

In [2]:
config

Config (path: /data/xiaowjia/Friends_fNIRS/work_dir/baseline_TFe-ds=PictureRating_t-d=1-nh=3-bs=128-ls=0.015-label=hand/tramsformer_num_heads3_dataset_name=PictureRating_label=hand_base_lr=1.5e-2_depth=1_batch_size=128.py): {'data_root': '/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', 'work_dir': '/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/Friends_fNIRS_Fuzzy/work_dir', 'dataset_name': 'PictureRating', 'dataset_para': {'type': 'fNIRS_Emo_CL_all_between_subject', 'data_root': '/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset/step1_PPCS_0.01_0.2PictureRating_data.pkl', 'test_size': 0.3, 'dataset_random_state': 42, 'temp_save_folder': '/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset/temp', 'label': 'hand', 'subject_split': 'between'}, 'base_lr': 0.015, 'batch_size': 128, 'lr': 0.015, 'model_para': {'img_size': (40, 33), 'num_heads': 3, 'depth': 1, 'cl_embed_dim': 64, 'encoder_type': 'Transformer', 'dropout': 0.3, 'model_ckpt': Non

In [ ]:
config_path = '/data/xiaowjia/Friends_fNIRS/work_dir/AFTF-ds=PictureClass_l-d=1-nh=5-r=1-bs=128-ls=0.003-label=hand/FuzzyTramsformer_ALL_num_rules1_num_heads5_dataset_name=PictureClass_label=hand_base_lr=3e-3_depth=1_batch_size=128.py'
config = load_config(config_path)

In [ ]:
config.data['train']['data_root'] = config.data['train']['data_root'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
config.data['val']['data_root'] = config.data['val']['data_root'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
config.data['test']['data_root'] = config.data['test']['data_root'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')

config.data['train']['temp_save_folder'] = config.data['train']['temp_save_folder'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
config.data['val']['temp_save_folder'] = config.data['val']['temp_save_folder'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
config.data['test']['temp_save_folder'] = config.data['test']['temp_save_folder'].replace('/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset', '/data/xiaowjia/Friends_fNIRS/data')
'/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset/temp/data_table_between_train.csv'
'/projects/CIBCIGroup/00DataUploading/Liang/fuzzy_fnirs/dataset'



In [ ]:
print(config.data['train']['data_root'])
print(config.data['val']['data_root'])
print(config.data['test']['data_root'])
print(config.data['train']['temp_save_folder'])
print(config.data['val']['temp_save_folder'])
print(config.data['test']['temp_save_folder'])